In [28]:
import os


In [29]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Github Actions A4'

In [32]:
os.chdir('.\Text_Summarization')

In [33]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Github Actions A4\\Text_Summarization'

In [56]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfiguration:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [57]:
from Text_Summarizer.constants import *
from Text_Summarizer.utils.common import read_yaml, create_directories

In [58]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):    
        self.config= read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        create_directories([self.config.artifacts])

    def get_data_ingestion_config(self)-> DataIngestionConfiguration:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfiguration(
            root_dir=config.root_dir,
            source_url =config.source_url,
            local_data_file =config.local_data_file,
            unzip_dir =config.unzip_dir
        )
        return data_ingestion_config

In [59]:
import os
import urllib.request as request
import zipfile
from Text_Summarizer.logging import logger
from Text_Summarizer.utils.common import get_size


In [66]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfiguration):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [67]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-15 21:23:20,719: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-15 21:23:20,721: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-15 21:23:20,722: INFO: common: craeted directory at : artifacts]
[2024-04-15 21:23:20,725: INFO: common: craeted directory at : artifacts/data_ingestion]
[2024-04-15 21:23:30,875: INFO: 2349157261: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E50C:FD079:59E68:6D2EC:661D4D71
Accept-Ranges: bytes
Date: Mon, 15 Apr 2024 15:53:24 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10250-MAA
X-Cache: MI